D-Waveマシンに接続するためのライブラリdwave-ocean-sdkをインポートします。

In [14]:
pip install dwave-ocean-sdk


今回はとりあえず、D-Waveマシンを使うことが目的なので、ランダムで10×10のQUBO行列を作成します。

In [15]:
import numpy as np

N = 10
QUBO = np.random.randn(N**2).reshape(N,N)

np.random.randn()は、平均0、分散1のガウス分布に従う乱数を生成するという関数です。 
print()でQUBO行列の中身を見れるよ。

In [16]:
print(QUBO[0][1])

0.7990714377279922


「20211213_第１回量子コンピュータ勉強会～量子コンピュータってなんだろう？まずは触ってみよう～ .pdf」の手順にしたがって、D-Waveのトークンを取得してください。
下記のXXXXXXXXXXXXXXの部分を取得したトークンに差し替えて下さい。

In [17]:
token = 'XXXXXXXXXXXXXX'  # 個人のAPI tokenを使用
endpoint = 'https://cloud.dwavesys.com/sapi/'


これでD-Wave Systems社の量子アニーリングマシンを利用する環境が整いました。 続けて量子アニーリングマシンを呼び出す関数をいくつか用意します。

In [18]:

from dwave.system import DWaveSampler, EmbeddingComposite

dw_sampler = DWaveSampler(solver='Advantage_system4.1', token=token, endpoint=endpoint)

DWaveSamplerは、量子アニーリングマシンにアクセスするための関数です。 solver=というオプションで、どの量子アニーリングマシンを使うのかを指定することができます。

利用できるソルバーは、トークンを発行したD-Waveのダッシュボードで確認できます。

solver = 'Advantage_system4.1'とします。

量子アニーリングマシンの中にはQPU（Quantum Processing Unit）と呼ばれる部品があり、 そこに搭載された超伝導量子ビットによる回路のパラメータを調整して$Q_{ij}$を与えます。 回路の設計上、$Q_{ij}$で指定された構造を作ることができない場合には、他の回路や量子ビットをうまく組み合わせるエンベッドという作業をします。 その作業をするには先ほど読み込んだ関数のうちEmbeddingComposite()という関数を利用することができます。

In [19]:
sampler = EmbeddingComposite(dw_sampler)


エンべディングを終えると、超伝導量子ビットにどの$x_i$を割り当てるか、 どのように$Q_{ij}$を割り当てるのかが決められた新しいsamplerに置き換わります。 これを利用することで,先ほど用意したQUBO行列に基づく組合せ最適化問題を量子アニーリングマシンに解かせることができます。

In [20]:
sampleset = sampler.sample_qubo(QUBO, num_reads=10)


sampler.sample_quboというのが、用意したsamplerの機能のうちQUBO形式の組合せ最適化問題を解く関数です。この引数として、先ほど用意したQUBO行列を入れます。 num_readsは非常にユニークなオプションで、何度結果を読み出すかを指定するところです。 結果を受け取ったsamplesetには、指定した組合せ最適化問題をnum_readsで指定した回数分解いた答えが格納されています。

In [21]:
print(sampleset.record)


[([1, 0, 1, 0, 0, 1, 0, 1, 0, 1], -11.36653819, 2, 0.)
 ([0, 0, 0, 0, 1, 1, 0, 1, 1, 1], -11.33730423, 4, 0.)
 ([1, 0, 1, 0, 1, 1, 0, 1, 1, 1], -11.27184774, 1, 0.)
 ([0, 0, 1, 0, 1, 1, 0, 1, 1, 1], -11.1198523 , 1, 0.)
 ([1, 0, 0, 0, 1, 1, 0, 1, 0, 1], -10.44468558, 1, 0.)
 ([0, 1, 0, 0, 1, 1, 0, 1, 1, 1], -10.37974772, 1, 0.)]


結果の一番目を取り出したいときはsampleset.record[0]として、 $x_i$などの結果を知りたいときは、sampleset.record[0][0]から取り出すことができます。 さらにコスト関数の値を知りたいときはsampleset.record[0][1] その答えが何度出てきたのか頻度を知りたいときはsampleset.record[0][2]で調べることができます。

In [22]:
sampleset.record[0][0]


array([1, 0, 1, 0, 0, 1, 0, 1, 0, 1], dtype=int8)

「量子アニーリングマシンを触ったぜ！」って友達に自慢しましょう！！